In [1]:
import os
import pandas as pd
import datetime
import time
import numpy as np

In [2]:
pd.set_option('display.max_rows',500)
data_file_name = 'line_oa_chat_csv_230717_091224'

In [5]:
os.chdir('C:\\Users\\User\\Desktop\\Jing-Ying\\競賽\\iTrush\\第二階段實作\\出題單位提供資料')
os.getcwd()

'C:\\Users\\User\\Desktop\\Jing-Ying\\競賽\\iTrush\\第二階段實作\\出題單位提供資料'

# 讀取所有對話資料及整理

In [34]:
start = time.time()

# 讀入所有資料、合併為一個檔案，並設立檔案的INDEX
col_name = ['傳送者類型','傳送者名稱','傳送日期','傳送時間','內容']
final_col = ['ttl_index','csv_num','date_time','week_day','diff_hour','dialogue_num','sentence_num'] + col_name

week_list = ["星期一","星期二","星期三","星期四","星期五","星期六","星期日"]
all_df = pd.DataFrame(columns = final_col)

print('提供檔案數:',len(os.listdir(os.getcwd() + '\\' + data_file_name)))
for i in os.listdir(os.getcwd() + '\\' + data_file_name):
    tmp_df = pd.read_csv(data_file_name + '\\' + i ,names = col_name,encoding='utf-8',engine='python').iloc[4:,:]
    
    # 新增檔案index欄位
    tmp_df['csv_num'] = [i.split('_')[0]] * tmp_df.shape[0]
    
    # 新增對話時間的星期欄位
    tmp_df['week_day'] = tmp_df[['傳送日期']].applymap(lambda x :week_list[datetime.date(int(x.split('/')[0]),int(x.split('/')[1]), int(x.split('/')[2])).weekday()])
    
    # 合併日期與時間
    tmp_df['date_time'] = tmp_df['傳送日期'] + ' ' + tmp_df['傳送時間']
    tmp_df['date_time'] = tmp_df[['date_time']].applymap(lambda x : datetime.datetime.strptime(x.replace('/','-'), '%Y-%m-%d %H:%M:%S'))
    
    # 計算該發話與前一個發話的時間差距
    tmp_df1 = tmp_df.copy()
    tmp_df1.index = [i+1 for i in list(tmp_df.index)]
    tmp_df1.columns = ['傳送者類型', '傳送者名稱', '傳送日期', '傳送時間', '內容', 'csv_index', 'week_day', 'date_time_lasttime']
    tmp_df = pd.concat([tmp_df, tmp_df1[['date_time_lasttime']]], axis=1).iloc[:-1,:]
    tmp_df['diff_time'] = (tmp_df['date_time'] - tmp_df['date_time_lasttime']).fillna(0)
    tmp_df['diff_hour'] = tmp_df.diff_time.map(lambda x : round(x.total_seconds()/3600),2)
    
    # 新增同客戶中對話數的判斷(暫定超過24小時即為新對話)，及新增同一個對話中的流水號
    dialogue_index = list([0])
    sentence_index = list([0])
    for i in tmp_df.diff_hour.tolist()[1:]:
        if i > 24:
            dialogue_index.append(dialogue_index[-1] + 1)
            sentence_index.append(0)
        else:
            dialogue_index.append(dialogue_index[-1])
            sentence_index.append(sentence_index[-1]+1)
    tmp_df['dialogue_num'] = dialogue_index
    tmp_df['sentence_num'] = sentence_index
    
    # 產出對話最終的index
    tmp_df['ttl_index'] = tmp_df['csv_num'].astype(str) + '_' + tmp_df['dialogue_num'].astype(str)
    
    # 選取需要的欄位
    tmp_df = tmp_df[final_col]
    
    # 合併資料
    all_df = all_df.append(tmp_df).reset_index(drop=True)

all_df = all_df[all_df.ttl_index != 'nan_0'].reset_index(drop=True)
print('讀入檔案數:',len(set(all_df.csv_num)))


end = time.time()

# 輸出結果
print("執行時間：%f 分鐘" % ((end - start)/60))

提供檔案數: 803
讀入檔案數: 335
執行時間：1.275307 分鐘


In [38]:
print(all_df[['date_time']].sort_values(by=['date_time']).head(1).values[0][0])
print(all_df[['date_time']].sort_values(by=['date_time']).tail(1).values[0][0])

2022-07-17 16:20:51
2023-07-17 09:05:40


In [40]:
all_df.shape

(11955, 12)

In [41]:
all_df[(all_df.傳送者類型 == 'User')].shape

(4730, 12)

In [148]:
# user_df[['內容','ttl_index']].groupby('內容').count().sort_values(by=['ttl_index'], ascending=False)

In [149]:
no_sentence = ['照片已傳送','貼圖已傳送','謝謝','你好','您好','感謝','謝謝您','對','好','好的','沒有']
user_df = all_df[(all_df.傳送者類型 == 'User') & ~(all_df.內容.isin(no_sentence))].reset_index(drop=True)

In [150]:
user_df.shape

(2597, 12)